Documentation: https://devportal.api-prod.route53.lexis.com/api?spec=NewsAPI.yml

In [3]:
from lexisnexisapi import webservices
from lexisnexisapi import credentials

In [4]:
# NOTE: set the credential at the path specified in the function call below
credentials.cred_file_path()

print(f"Credentials located at {credentials.cred_file_path()}")
# # check if credentials are set
# credentials.get_Credentials()


Credentials located at /Users/danielliu/.lnapi/.cred


In [5]:
# load credentials into lexisnexis api
token = webservices.token()

*Refer to the LexisNexis Developer Portal: [News API](https://dev.lexisnexis.com/dev-portal/documentation/News#/News%20API/get_News) for guidance on using parameters \$search, \$expand, \$top, and \$orderby.*

Note: The section is in Web Services API -> Content Type -> News -> API.
Also has examples: API ->  e. Note:

In [6]:
#Replace Substitute  with your desired search term(s)
search_string = '"Olympics 2024"'    #Use quotation marks "search string" if the search string is a phrase, e.g.,"Olympics 2024"

parameters ={
            '$search':search_string,
            '$expand':'Document', #A navigation property name which will be included with the current result set.
            # '$top':'5',  #Sets the maximum number of results to receive for this request.
            # #Filter with two conditions
            '$filter': "Language eq LexisNexis.ServicesApi.Language'English' and year(Date) eq 2024",
            '$select': 'ResultId, Title, Source',
            '$orderby': 'Date asc'
           }

# NOTE: the "$expand: Document" parameter is required to get the full document's HTML.
# However, if the query limit is reached, the "$expand: Document" parameter will cause a 429 error
# (i.e. if you remove it, the query will work but you will only get the metadata of the documents).

In [7]:
from requests import HTTPError
try:
    data = webservices.call_api(access_token=token,endpoint='News',params=parameters)  #Set endpoint='News'
    data # response in json
except HTTPError as e:
    print("STATUS CODE")
    print(e.response.status_code)


In [8]:
print(data)

{'@odata.context': 'https://services-api.lexisnexis.com/v1/$metadata#News(ResultId,Title,Source,Document)', '@odata.count': 10838, 'value': [{'ResultId': 'urn:contentItem:6B0N-N521-JBN4-B14G-00000-00', 'Title': 'Savita Punia to lead Indian womens hockey squad in Olympic Qualifiers', 'Document': {'DocumentId': '/shared/document/news/urn:contentItem:6B0N-N521-JBN4-B14G-00000-00', 'DocumentIdType': 'DocFullPath', 'Content': '<entry xmlns="http://www.w3.org/2005/Atom"><id>urn:contentItem:6B0N-N521-JBN4-B14G-00000-00</id><title>Savita Punia to lead Indian womens hockey squad in Olympic Qualifiers</title><published>2024-01-01T00:00:00Z</published><updated>2024-11-14T18:23:57Z</updated><author><name>LexisNexis</name></author><content type="application/xml"><!--Transformation version 1.25--><articleDoc xmlns="" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://www.lexisnexis.com/xmlschemas/content/public/articledoc/1/" schemaVersion="1.8" xml:lang="en"

In [9]:
for k in data["value"][2].keys():
    print(k, ":", data["value"][2][k])

ResultId : urn:contentItem:6B0S-MM61-F0JP-W2XT-00000-00
Title : SPORT GALORE IN '24
Document : {'DocumentId': '/shared/document/news/urn:contentItem:6B0S-MM61-F0JP-W2XT-00000-00', 'DocumentIdType': 'DocFullPath', 'Content': '<entry xmlns="http://www.w3.org/2005/Atom"><id>urn:contentItem:6B0S-MM61-F0JP-W2XT-00000-00</id><title>SPORT GALORE IN \'24</title><published>2024-01-01T00:00:00Z</published><updated>2024-11-14T18:23:57Z</updated><author><name>LexisNexis</name></author><content type="application/xml"><!--Transformation version 1.25--><articleDoc xmlns="" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://www.lexisnexis.com/xmlschemas/content/public/articledoc/1/" schemaVersion="1.8" xml:lang="en"><articleDocHead><itemInfo/></articleDocHead><nitf:body xmlns:nitf="http://iptc.org/std/NITF/2006-10-18/"><nitf:body.head><nitf:hedline><nitf:hl1>SPORT GALORE IN \'24</nitf:hl1></nitf:hedline><nitf:byline><author><person><nameText>JULIAN LINDEN</name

In [10]:
top_result_data = data["value"][0]["Document"]["Content"]
top_result_data

'<entry xmlns="http://www.w3.org/2005/Atom"><id>urn:contentItem:6B0N-N521-JBN4-B14G-00000-00</id><title>Savita Punia to lead Indian womens hockey squad in Olympic Qualifiers</title><published>2024-01-01T00:00:00Z</published><updated>2024-11-14T18:23:57Z</updated><author><name>LexisNexis</name></author><content type="application/xml"><!--Transformation version 1.25--><articleDoc xmlns="" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://www.lexisnexis.com/xmlschemas/content/public/articledoc/1/" schemaVersion="1.8" xml:lang="en"><articleDocHead><itemInfo/></articleDocHead><nitf:body xmlns:nitf="http://iptc.org/std/NITF/2006-10-18/"><nitf:body.head><nitf:hedline><nitf:hl1>Savita Punia to lead Indian womens hockey squad in Olympic Qualifiers</nitf:hl1></nitf:hedline><nitf:dateline><location>New Delhi, 2024-01-01 00:28:45</location></nitf:dateline></nitf:body.head><nitf:body.content><bodyText><p nitf:lede="true"> Bengaluru Veteran goalkeeper Savita

# Display

In [11]:
from IPython.core.display import display, HTML
display(HTML(top_result_data))

/var/folders/39/kgsx4rgd2sj8dwffgf80cfgr0000gq/T/ipykernel_94294/625531025.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [12]:
from bs4 import BeautifulSoup

BS = BeautifulSoup(top_result_data, "lxml")

text = "\n".join([p.get_text(" ", strip=True) for p in BS.find_all("p")])
print(text)

Bengaluru Veteran goalkeeper Savita Punia will continue to lead the side and Vandana Katariya will be her deputy as Hockey India on Saturday named an 18member Indian womens team for the Olympic Qualifiers scheduled to be held in Ranchi from January 13 to 19 India will take the field in Ranchi looking to finish as one of the top three teams in the competition and secure qualification for the Paris Olympics 2024 The FIH Hockey Olympic Qualifiers Ranchi is an important tournament on our Paris Olympics journey It is imperative that we live up to the expectations and all the players in the squad are working to secure qualification Indian Womens chief coach Janneke Schopman said in a release After careful consideration we have chosen a wellbalanced team with immense skill and experience in all departments Also Savita and Vandana have faced highpressure situations numerous times in their career so far They are well equipped to guide the rest of the team as the Captain and ViceCaptain she adde